## Tickers to follow
The objective here is to identify which tickers are available on stocktwits. Then I can scrape the watchers daily to see if there are any stocks that have a big increase.

In [1]:
import numpy as np
import pandas as pd
from os import system
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import config
import matplotlib.pyplot as plt

In [ ]:
nsdq = pd.read_csv('nsdq.csv')

In [ ]:
nsdq_tickers = nsdq['Watchlist Scanner'][3:].values.tolist()

In [ ]:
# Stocktwits login function 
def stocktwits_login():

    pd.set_option('display.max_columns', 20)
    pd.set_option('display.max_colwidth', 200)
    mobile_emulation = { "deviceName": "iPhone X" }

    chrome_options = Options()
    chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
    driver = webdriver.Chrome(chrome_options = chrome_options)

    driver.get('https://stocktwits.com/')
    
    time.sleep(.5)
    login = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[2]/div/div/div/div/div[1]/div[1]/div[4]/div/span[2]')
    login.click()

    #username
    username = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[4]/div[2]/div/form/div[1]/div[1]/label/input')
    username.click()
    username.send_keys(config.username)

    #password
    password = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[4]/div[2]/div/form/div[1]/div[2]/label/input')
    password.click()
    password.send_keys(config.password)

    #sign in 
    signin = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[4]/div[2]/div/form/div[2]/div[1]/button')
    signin.click()
    
    return driver

In [ ]:
# Takes in Ticker List from wikipedia page 
# Scrapes all of the watchers for each stock in the S&P 500 
# Looking for stocks with over 10k watchers to make sure the stock gets enough activity 
# because stocks with no activity have huge sentiment swings which is not good for analysis 
def stocks_500_watch(ticker_list):
    driver = stocktwits_login()
    watch_dict = {}
    
    for i in ticker_list:
        
        driver.get(f'https://stocktwits.com/symbol/{i}')
        try:
            watchers = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[2]/div[2]/div[1]/div/div/div/div/div[2]/div/strong')
            watchers = watchers.text
            watch_dict[i] = watchers
        except:
            continue
        time.sleep(.7)
        
    driver.quit()
        
    return watch_dict

In [ ]:
driver = stocktwits_login()

In [ ]:
# Scrape 1000 at a time to avoid issues or loss 
nsdq_dictionary = stocks_500_watch(nsdq_tickers[6000:])

In [ ]:
# nsdq_dict = {}

In [ ]:
# Final Dictionary 
nsdq_dict.update(nsdq_dictionary)

In [ ]:
df = pd.DataFrame.from_dict(nsdq_dict, orient="index")

In [ ]:
df.shape

In [ ]:
df.to_csv('nsdq_watchers.csv')